### Bibliotecas

In [1]:
import os
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import openpyxl
from statsmodels.tsa.stattools import ccf
from openpyxl.styles import PatternFill


### Funções para leitura DF, resample por 30 min, correlações horarias simples

In [2]:
def resample_30_minutos(df, caminho_saida):
    """
    Função para realizar o resample dos dados em intervalos de 30 minutos e salvar em um arquivo CSV.
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - caminho_saida: Caminho do arquivo CSV de saída.
        
    Retorna:
        - DataFrame resample.
    """
    df_resampled = df.resample('30T').median()
    df_resampled.to_csv(caminho_saida)
    return df_resampled

def calcular_correlacoes(df, window_size=3, inicio='2024-01-29 09:30:00'):
    """
    Função para calcular as correlações em janelas deslizantes.
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - window_size: Tamanho da janela deslizante para calcular as correlações (padrão é 3).
        - inicio: Data e hora de início para filtrar o DataFrame (padrão é '2024-01-29 09:30:00').
        
    Retorna:
        - DataFrame contendo as correlações calculadas.
    """
    df_resampled_half_hour = df.resample('30T').median()
    correlations_list = []
    datetime_list = []
    df_resampled_half_hour_filtered = df_resampled_half_hour.loc[inicio:]

    for i in range(len(df_resampled_half_hour_filtered) - window_size + 1):
        window_index = range(i, i + window_size)
        df_window = df_resampled_half_hour_filtered.iloc[window_index]
        correlations = df_window.corr().iloc[0, 1:]
        correlations_ordered = correlations.reindex([
            'G2_Oleo_Motor_Temp', 'G2_Oleo_Motor_Press',
            'G2_Carga_Temperature', 'G2_Aumento_Pressão_Valvula_Real',
            'G2_Gerador_Corrente_Média', 'G2_Gerador_Tensão_Média',
            'G2_Gerador_Potência_Reativa', 'G2_Gerador_Potência_Aparente',
            'G2_Gerador_frequencia', 'G2_Gerador_Fator_de_Potência',
            'G2_Acelerador_Valv_Posição', 'G2_Jacket_Water_pressão',
            'G2_Jacket_Water_Temp', 'G2_Cilindro_EX_GAS_TEMP_Média',
            'g2_Turbocharger_Bypass_Posição', 'G2_Saída_Ativa_total',
            'G2_Saída_Reativa_total', 'G2_LAMBDA_TECJET', 'G2_Gerador_Corrente_L1',
            'G2_Gerador_Voltagem_L1_N', 'G2_Gerador_enrolamento_temp_L1',
            'G2_Gerador_enrolamento_temp_L2', 'G2_Gerador_enrolamento_temp_L3',
            'G2_Gerador_Extremidade_acionamento_rolamento',
            'G2_Gerador_Bearing_nondriveend', 'G2_Gerador_Corrente_L2',
            'G2_Gerador_Voltagem_L2_L3', 'G2_Gerador_Voltagem_L2_N',
            'G2_Gerador_Corrente_N', 'G2_Excitação_Voltagem',
            'G2_Gerador_Corrente_L3', 'G2_Gerador_Voltagem_L3_L1',
            'G2_Gerador_Voltagem_L3_N'
        ])
        correlations_list.append(correlations_ordered.values)
        datetime_list.append(df_window.index[-1])

    df_correlation_hourly = pd.DataFrame(correlations_list, columns=correlations_ordered.index)
    df_correlation_hourly['Datetime'] = datetime_list
    cols = df_correlation_hourly.columns.tolist()
    cols = [cols[-1]] + cols[:-1]
    df_correlation_hourly = df_correlation_hourly[cols]

    return df_correlation_hourly

def preencher_nan(df):
    return df.where(df.notnull(), np.nan)

def salvar_csv_com_dia_da_semana(df, caminho_saida):
    """
    Função para salvar o DataFrame em um arquivo CSV com uma coluna adicional para o dia da semana.
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - caminho_saida: Caminho do arquivo CSV de saída.
    """
    df['Dia_Semana'] = df['Datetime'].dt.strftime('%A')
    df.to_csv(caminho_saida, index=False, na_rep='NaN')

def salvar_pkl(df, caminho_saida):
    """
    Função para salvar o DataFrame em um arquivo pickle (.pkl).
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - caminho_saida: Caminho do arquivo pickle de saída.
    """
    df.to_pickle(caminho_saida)

### Chamadas das Funções

In [3]:
# Caminho do arquivo PKL
caminho_arquivos_pkl_29_01_and_20_05 = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_concatenados\files_29_01_and_20_05_concatenados_sem_NaNs.pkl'

# Leitura do arquivo PKL
df_parte_I = pd.read_pickle(caminho_arquivos_pkl_29_01_and_20_05)

# Caminho do arquivo CSV de saída para o resample de 30 minutos
caminho_arquivo_csv_resampled_1 = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\resample_30_minutos.csv'
# Caminho do arquivo PKL de saída para o resample de 30 minutos
caminho_arquivo_pkl_resampled_1 = caminho_arquivo_csv_resampled_1.replace('.csv', '.pkl')

# Chamada da função para realizar o resample e salvar em um arquivo CSV
df_resampled_1 = resample_30_minutos(df_parte_I, caminho_arquivo_csv_resampled_1)

# Chamada da função para salvar o resample em um arquivo PKL
salvar_pkl(df_resampled_1, caminho_arquivo_pkl_resampled_1)

# Chamada da função para calcular as correlações
df_correlation_hourly = calcular_correlacoes(df_parte_I)

# Preenchimento dos valores ausentes com NaN
df_correlation_hourly = preencher_nan(df_correlation_hourly)

# Caminho do novo arquivo CSV para as correlações com o dia da semana
caminho_novo_arquivo_csv_com_dia_da_semana = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\resample_30_minutos_correlacoes_horarias_com_dia_da_semana.csv'

# Chamada da função para salvar o DataFrame das correlações com o dia da semana em um arquivo CSV
salvar_csv_com_dia_da_semana(df_correlation_hourly, caminho_novo_arquivo_csv_com_dia_da_semana)

### Funções para leitura DF, resample por 1 min, correlações minuto simples

In [4]:
def resample_1_minuto(df, caminho_saida):
    """
    Função para realizar o resample dos dados em intervalos de 1 minuto e salvar em um arquivo CSV.
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - caminho_saida: Caminho do arquivo CSV de saída.
    """
    df_resampled = df.resample('1T').median()
    df_resampled.to_csv(caminho_saida)
    return df_resampled

def calcular_correlacoes(df, window_size=3, inicio='2024-01-29 09:30:00'):
    """
    Função para calcular as correlações em janelas deslizantes.
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - window_size: Tamanho da janela deslizante para calcular as correlações (padrão é 3).
        - inicio: Data e hora de início para filtrar o DataFrame (padrão é '2024-01-29 09:30:00').
        
    Retorna:
        - DataFrame contendo as correlações calculadas.
    """
    df_resampled_one_minute = df.resample('1T').median()
    correlations_list = []
    datetime_list = []
    df_resampled_one_minute_filtered = df_resampled_one_minute.loc[inicio:]

    for i in range(len(df_resampled_one_minute_filtered) - window_size + 1):
        window_index = range(i, i + window_size)
        df_window = df_resampled_one_minute_filtered.iloc[window_index]
        correlations = df_window.corr().iloc[0, 1:]
        correlations_ordered = correlations.reindex([
            'G2_Oleo_Motor_Temp', 'G2_Oleo_Motor_Press',
            'G2_Carga_Temperature', 'G2_Aumento_Pressão_Valvula_Real',
            'G2_Gerador_Corrente_Média', 'G2_Gerador_Tensão_Média',
            'G2_Gerador_Potência_Reativa', 'G2_Gerador_Potência_Aparente',
            'G2_Gerador_frequencia', 'G2_Gerador_Fator_de_Potência',
            'G2_Acelerador_Valv_Posição', 'G2_Jacket_Water_pressão',
            'G2_Jacket_Water_Temp', 'G2_Cilindro_EX_GAS_TEMP_Média',
            'g2_Turbocharger_Bypass_Posição', 'G2_Saída_Ativa_total',
            'G2_Saída_Reativa_total', 'G2_LAMBDA_TECJET', 'G2_Gerador_Corrente_L1',
            'G2_Gerador_Voltagem_L1_N', 'G2_Gerador_enrolamento_temp_L1',
            'G2_Gerador_enrolamento_temp_L2', 'G2_Gerador_enrolamento_temp_L3',
            'G2_Gerador_Extremidade_acionamento_rolamento',
            'G2_Gerador_Bearing_nondriveend', 'G2_Gerador_Corrente_L2',
            'G2_Gerador_Voltagem_L2_L3', 'G2_Gerador_Voltagem_L2_N',
            'G2_Gerador_Corrente_N', 'G2_Excitação_Voltagem',
            'G2_Gerador_Corrente_L3', 'G2_Gerador_Voltagem_L3_L1',
            'G2_Gerador_Voltagem_L3_N'
        ])
        correlations_list.append(correlations_ordered.values)
        datetime_list.append(df_window.index[-1])

    df_correlation_2_minutes = pd.DataFrame(correlations_list, columns=correlations_ordered.index)
    df_correlation_2_minutes['Datetime'] = datetime_list
    cols = df_correlation_2_minutes.columns.tolist()
    cols = [cols[-1]] + cols[:-1]
    df_correlation_2_minutes = df_correlation_2_minutes[cols]

    return df_correlation_2_minutes

def preencher_nan(df):
    return df.where(df.notnull(), np.nan)

def salvar_csv_com_dia_da_semana(df, caminho_saida):
    """
    Função para salvar o DataFrame em um arquivo CSV com uma coluna adicional para o dia da semana.
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - caminho_saida: Caminho do arquivo CSV de saída.
    """
    df['Dia_Semana'] = df['Datetime'].dt.strftime('%A')
    df.to_csv(caminho_saida, index=False, na_rep='NaN')

def salvar_pkl(df, caminho_saida):
    """
    Função para salvar o DataFrame em um arquivo pickle (.pkl).
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - caminho_saida: Caminho do arquivo pickle de saída.
    """
    df.to_pickle(caminho_saida)

### Chamadas das Funções

In [5]:
# Caminho do arquivo PKL
caminho_arquivos_pkl_29_01_and_20_05 = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_concatenados\files_29_01_and_20_05_concatenados_sem_NaNs.pkl'

# Leitura do arquivo PKL
df_parte_I = pd.read_pickle(caminho_arquivos_pkl_29_01_and_20_05)

# Caminho do arquivo CSV de saída para o resample de 1 minuto
caminho_arquivo_csv_resampled_1_minuto = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\resample_1_minuto.csv'
# Caminho do arquivo PKL de saída para o resample de 30 minutos
caminho_arquivo_pkl_resampled_2 = caminho_arquivo_csv_resampled_1_minuto.replace('.csv', '.pkl')

# Chamada da função para realizar o resample de 1 minuto e salvar em um arquivo CSV
df_resampled_2= resample_1_minuto(df_parte_I, caminho_arquivo_csv_resampled_1_minuto)

# Chamada da função para salvar o resample em um arquivo PKL
salvar_pkl(df_resampled_2, caminho_arquivo_pkl_resampled_2)

# Chamada da função para calcular as correlações
df_correlation_2_minutes = calcular_correlacoes(df_parte_I)

# Preenchimento dos valores ausentes com NaN
df_correlation_2_minutes = preencher_nan(df_correlation_2_minutes)

# Caminho do novo arquivo CSV para as correlações com o dia da semana
caminho_novo_arquivo_csv_com_dia_da_semana = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\resample_1_minuto_correlacoes_2_minutos_com_dia_da_semana.csv'

# Chamada da função para salvar o DataFrame das correlações com o dia da semana em um arquivo CSV
salvar_csv_com_dia_da_semana(df_correlation_2_minutes, caminho_novo_arquivo_csv_com_dia_da_semana)

### Funções para leitura DF, resample por 5 minutos, correlações simples

In [6]:
def resample_5_minutos(df, caminho_saida):
    """
    Função para realizar o resample dos dados em intervalos de 1 minuto e salvar em um arquivo CSV.
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - caminho_saida: Caminho do arquivo CSV de saída.
    """
    df_resampled = df.resample('5T').median()
    df_resampled.to_csv(caminho_saida)
    return df_resampled

def calcular_correlacoes(df, window_size=3, inicio='2024-01-29 09:30:00'):
    """
    Função para calcular as correlações em janelas deslizantes.
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - window_size: Tamanho da janela deslizante para calcular as correlações (padrão é 3).
        - inicio: Data e hora de início para filtrar o DataFrame (padrão é '2024-01-29 09:30:00').
        
    Retorna:
        - DataFrame contendo as correlações calculadas.
    """
    df_resampled_five_minutes = df.resample('5T').median()
    correlations_list = []
    datetime_list = []
    df_resampled_five_minutes_filtered = df_resampled_five_minutes.loc[inicio:]

    for i in range(len(df_resampled_five_minutes_filtered) - window_size + 1):
        window_index = range(i, i + window_size)
        df_window = df_resampled_five_minutes_filtered.iloc[window_index]
        correlations = df_window.corr().iloc[0, 1:]
        correlations_ordered = correlations.reindex([
            'G2_Oleo_Motor_Temp', 'G2_Oleo_Motor_Press',
            'G2_Carga_Temperature', 'G2_Aumento_Pressão_Valvula_Real',
            'G2_Gerador_Corrente_Média', 'G2_Gerador_Tensão_Média',
            'G2_Gerador_Potência_Reativa', 'G2_Gerador_Potência_Aparente',
            'G2_Gerador_frequencia', 'G2_Gerador_Fator_de_Potência',
            'G2_Acelerador_Valv_Posição', 'G2_Jacket_Water_pressão',
            'G2_Jacket_Water_Temp', 'G2_Cilindro_EX_GAS_TEMP_Média',
            'g2_Turbocharger_Bypass_Posição', 'G2_Saída_Ativa_total',
            'G2_Saída_Reativa_total', 'G2_LAMBDA_TECJET', 'G2_Gerador_Corrente_L1',
            'G2_Gerador_Voltagem_L1_N', 'G2_Gerador_enrolamento_temp_L1',
            'G2_Gerador_enrolamento_temp_L2', 'G2_Gerador_enrolamento_temp_L3',
            'G2_Gerador_Extremidade_acionamento_rolamento',
            'G2_Gerador_Bearing_nondriveend', 'G2_Gerador_Corrente_L2',
            'G2_Gerador_Voltagem_L2_L3', 'G2_Gerador_Voltagem_L2_N',
            'G2_Gerador_Corrente_N', 'G2_Excitação_Voltagem',
            'G2_Gerador_Corrente_L3', 'G2_Gerador_Voltagem_L3_L1',
            'G2_Gerador_Voltagem_L3_N'
        ])
        correlations_list.append(correlations_ordered.values)
        datetime_list.append(df_window.index[-1])

    df_correlation_10_minutes = pd.DataFrame(correlations_list, columns=correlations_ordered.index)
    df_correlation_10_minutes['Datetime'] = datetime_list
    cols = df_correlation_10_minutes.columns.tolist()
    cols = [cols[-1]] + cols[:-1]
    df_correlation_10_minutes = df_correlation_10_minutes[cols]

    return df_correlation_10_minutes

def preencher_nan(df):
    return df.where(df.notnull(), np.nan)

def salvar_csv_com_dia_da_semana(df, caminho_saida):
    """
    Função para salvar o DataFrame em um arquivo CSV com uma coluna adicional para o dia da semana.
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - caminho_saida: Caminho do arquivo CSV de saída.
    """
    df['Dia_Semana'] = df['Datetime'].dt.strftime('%A')
    df.to_csv(caminho_saida, index=False, na_rep='NaN')

def salvar_pkl(df, caminho_saida):
    """
    Função para salvar o DataFrame em um arquivo pickle (.pkl).
    
    Parâmetros:
        - df: DataFrame contendo os dados.
        - caminho_saida: Caminho do arquivo pickle de saída.
    """
    df.to_pickle(caminho_saida)

### Chamadas das Funções

In [7]:
# Caminho do arquivo PKL
caminho_arquivos_pkl_29_01_and_20_05 = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_concatenados\files_29_01_and_20_05_concatenados_sem_NaNs.pkl'

# Leitura do arquivo PKL
df_parte_I = pd.read_pickle(caminho_arquivos_pkl_29_01_and_20_05)

# Caminho do arquivo CSV de saída para o resample de 5 minutos
caminho_arquivo_csv_resampled_5_minutos = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\resample_5_minutos.csv'
# Caminho do arquivo PKL de saída para o resample de 5 minutos
caminho_arquivo_pkl_resampled_3 = caminho_arquivo_csv_resampled_5_minutos.replace('.csv', '.pkl')

# Chamada da função para realizar o resample de 5 minutos e salvar em um arquivo CSV
df_resampled_3 = resample_5_minutos(df_parte_I, caminho_arquivo_csv_resampled_5_minutos)

# Chamada da função para salvar o resample em um arquivo PKL
salvar_pkl(df_resampled_3, caminho_arquivo_pkl_resampled_3)

# Chamada da função para calcular as correlações
df_correlation_10_minutes = calcular_correlacoes(df_parte_I)

# Preenchimento dos valores ausentes com NaN
df_correlation_10_minutes = preencher_nan(df_correlation_10_minutes)

# Caminho do novo arquivo CSV para as correlações com o dia da semana
caminho_novo_arquivo_csv_com_dia_da_semana = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\resample_5_minutos_correlacoes_10_minutos_com_dia_da_semana.csv'

# Chamada da função para salvar o DataFrame das correlações com o dia da semana em um arquivo CSV
salvar_csv_com_dia_da_semana(df_correlation_10_minutes, caminho_novo_arquivo_csv_com_dia_da_semana)

## COMPONENTES ADICIONAIS

### Resample para frames de 30 segundos e respetivo local de guarda

In [3]:
# Convertendo o índice para o formato datetime
df_parte_I.index = pd.to_datetime(df_parte_I.index)

# Resample para obter os dados em intervalos de 30 segundos e calcular a média
df_resampled = df_parte_I.resample('30S').median()

# Definindo o caminho do novo arquivo CSV
caminho_novo_arquivo_csv = r'H:\Path_Python\Dados_Python_2024\Lameirinho\Ficheiros_PKL_Saida\Arquivos_correlacao_Excel\resample_30_segundos.csv'

# Salvando o DataFrame resample em um novo arquivo CSV
df_resampled.to_csv(caminho_novo_arquivo_csv)